# Vertica columns definition updates

## Problem 

Following the migration of analytics data tables CTG Vertica database to the data lake, table and column documentation in the data lake catalog has not been updated. It would be a significant manual effort to recapture all the business definitions in the data lake data dictionary (catalog). 

Intuit uses Alation Data Catalog as their central meta-data repository. The CTG Vertica data definitions we exported from the Alation Data Catalog and made available as Google sheet. 

We want to leverage the legacy documentation now available in a Google sheet to programmatically update the data lake description for the migrated tables and columns to the data lake. There are three scenarios:

1. table moved to the data lake and with the same name as the corresponding Vertica table
2. table moved to the data lake under a new name - a table migration mapping existing in the form of a Google sheet identifying source and target table. 
3. columns may have been replicated in numerous different tables in the data lake, for which we may not have a table mapping

Scenario #1 was easily resolved using the Alation Catalog Sync feature available in Alation. This resulted in the update of 292 tables and their columns documentation. 

The Alation Catalog sync feature cannot be used where table names are different. In this case, we need to use a scripted approach, leveraging Python and Pandas dataframes, to copy column level definitions from Vertica document to data lake. This approach relies on matching column names across all tables - scenario #3. The estimate is that over 2800 column definitions can be updated using this approach.

Finally, table level descriptions will be migrated to the data lake based on the migration table map. This potentially add another 300+ data lake table descriptions. 


## Approach 

Creates following data frames: 
* `df_vertica_desc`: vertica data catalog exported descriptions (read from Google Sheet)
    * create a deduplicated version of column descriptions, retaining the longest description as the preferred column description
    * create a deduplicated version of table descriptions, also retaining the longest description
* `df_alation_catalog`: existing alation data catalog export, from which to extract table and column objects with null descriptions - to avoid overwrite existing decriptions

These two catalogs can be merged on the column name to associate the Vertica description to the data lake column of the same name.   

* `df_cg_datasets`: Data Solution Engineering is working in parallel to manually created documentation. Load the work in progress descriptions into a data set that will be used to remove conflicts from the merged Vertica / Alation data sets (above). 

* `df_new_definitions`: merge of vertica_desc and df_alation_catalog matched on column name, excluding objects already present in cg_datasets. Removing column descriptions created by DSE can be done with a merge left join that excludes matching records (i.e. where DSE table name is null) - that way, only the rows from the merge catalog that do not have a match in the cg datasets will remain.

Use a similar approach as above to bring in table leve definition for tables where names do not match, to create a data lake table level description update. 
* `df_dl_map`: Google sheet containing from / to data lake tables, to enable updating data lake table descriptions where table name does not match Vertica table name. These records will be appended to the  `df_new_definitions` data frame. 

Finally, create a process to export to CSV new definitions, creating on CSV file per data lake table, as per DSE data catalog update process.  

In [1]:
import pandas as pd
import numpy as np
import os

import gspread

from datetime import datetime, timedelta
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

# import plotly.graph_objs as go
# import plotly.express as px

# from plotly.offline import iplot

## Configurations

In [2]:
# Alation current data catalog - from local export files
CATALOG_LOCAL_PATH = "S:/Data/alation_data"

# Alation catalog upload Github project 
GITHUB = {"LOCAL_PATH": "S:/Development/github/intuit/cg_datasets/hive",
          "DATA_CATALOG_FOLDER_NAME": "datacatalog",
          "CATALOG_FILE_NAME_SUFFIX": "vertica-legacy"}

# generic descriptions to be ignored
VERTICA_DESC_EXCEPTIONS = [
    "7216 approved",
    "Please refer to Consumer_dm.dim_agent",
    "Field not populated with any data as of 11/13/2018"
]

In [3]:
# Google sheet KPI book
VERTICA_SPREADSHEET_ID ='<removed>'
VERTICA_SHEET = 'vertica_descriptions'

DL_MAP_SPREADSHEET_ID = '<removed>'
DL_MAP_SHEET = 'table_list'

GOOGLE_TOKEN_PATH = "./creds/"
SERVICE_ACCOUNT_FILE = os.path.join(GOOGLE_TOKEN_PATH,'<removed>.json')
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

print(SERVICE_ACCOUNT_FILE)
print(SCOPES)

./creds/cg-data-analytics-4aadfb13b1cf.json
['https://www.googleapis.com/auth/spreadsheets']


## Connection

In [4]:
# connect to service account and generate a Google Sheet client
CREDENTIALS = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, SCOPES)
gc = gspread.authorize(CREDENTIALS)

print(gc)

In [5]:
# connect workbooks
vertica_workbook = gc.open_by_key(VERTICA_SPREADSHEET_ID)
dl_map_workbook = gc.open_by_key(DL_MAP_SPREADSHEET_ID)

print(vertica_workbook)
print(dl_map_workbook)

<Spreadsheet 'vertica_descriptions_consolidated' id:1UsEVy62FB8Dd36uwPcRX6w4nCazP1JBSRjiN-mSOp48>
<Spreadsheet 'ctg_vertica_table_documentation_migration' id:1lKAtO52l1np5Bu78T3aR_LJIJk7Qwo9fDiGkvJkntgg>


### Define functions

In [6]:
def read_gsheet(book,sheet):
    worksheet = book.worksheet(sheet)
    table = worksheet.get_all_values()
    return pd.DataFrame(table[1:],columns = table[0])

### Main Execution

#### Read vertica to data lake migration data lake table map

In [7]:
df_dl_map = read_gsheet(dl_map_workbook, DL_MAP_SHEET)
df_dl_map['key'] = df_dl_map['CTG Vertica Key']

df_dl_map.shape

(826, 11)

In [8]:
df_dl_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826 entries, 0 to 825
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CTG Vertica Key         826 non-null    object
 1   CTG Vertica AWS schema  826 non-null    object
 2   CTG Vertica AWS table   826 non-null    object
 3   Data Lake Key           826 non-null    object
 4   Data Lake Schema        826 non-null    object
 5   Data Lake Table         826 non-null    object
 6   Same schema             826 non-null    object
 7   Same table              826 non-null    object
 8   Sync Type               826 non-null    object
 9   Status                  826 non-null    object
 10  key                     826 non-null    object
dtypes: object(11)
memory usage: 71.1+ KB


In [9]:
df_dl_map.head()

,CTG Vertica Key,CTG Vertica AWS schema,CTG Vertica AWS table,Data Lake Key,Data Lake Schema,Data Lake Table,Same schema,Same table,Sync Type,Status,key
0,backup_schema.tto_mapping_segment,backup_schema,tto_mapping_segment,tax_src.tto_mapping_segment,tax_src,tto_mapping_segment,FALSE,TRUE,Schema Level,Out of Scope,backup_schema.tto_mapping_segment
1,bup_dm.amex_fact_ad_performance,bup_dm,amex_fact_ad_performance,bup_dm.amex_fact_ad_performance,bup_dm,amex_fact_ad_performance,TRUE,TRUE,Schema Level,Out of Scope,bup_dm.amex_fact_ad_performance
2,bup_dm.cfg_etl_job,bup_dm,cfg_etl_job,bup_dm.cfg_etl_job,bup_dm,cfg_etl_job,TRUE,TRUE,Schema Level,Out of Scope,bup_dm.cfg_etl_job
3,bup_dm.dim_bup_ad_content,bup_dm,dim_bup_ad_content,bup_dm.dim_bup_ad_content,bup_dm,dim_bup_ad_content,TRUE,TRUE,Schema Level,Out of Scope,bup_dm.dim_bup_ad_content
4,bup_dm.dim_bup_agencies,bup_dm,dim_bup_agencies,bup_dm.dim_bup_agencies,bup_dm,dim_bup_agencies,TRUE,TRUE,Schema Level,Out of Scope,bup_dm.dim_bup_agencies


In [10]:
df_dl_map['same_table_name'] = df_dl_map['CTG Vertica AWS table'] == df_dl_map['Data Lake Table']

df_dl_map[['Data Lake Table','same_table_name']].groupby('same_table_name').count()

,Data Lake Table
same_table_name,
False,338
True,488


In [11]:
# retain tables with different names 
df_dl_map = df_dl_map[~df_dl_map.same_table_name]

df_dl_map.head()

,CTG Vertica Key,CTG Vertica AWS schema,CTG Vertica AWS table,Data Lake Key,Data Lake Schema,Data Lake Table,Same schema,Same table,Sync Type,Status,key,same_table_name
33,bup_source.src_impact_radius,bup_source,src_impact_radius,smartmoney_src.src_impact_mint,smartmoney_src,src_impact_mint,FALSE,FALSE,Column Level,,bup_source.src_impact_radius,False
34,bup_source.src_marcus,bup_source,src_marcus,bup_src.v_partner_marcus_etl,bup_src,v_partner_marcus_etl,FALSE,FALSE,Column Level,,bup_source.src_marcus,False
36,bup_source.src_rakuten,bup_source,src_rakuten,smartmoney_src.src_rakuten_mint,smartmoney_src,src_rakuten_mint,FALSE,FALSE,Column Level,,bup_source.src_rakuten,False
39,bup_stage.stg_rakuten,bup_stage,stg_rakuten,smartmoney_stg.stg_rakuten_mint,smartmoney_stg,stg_rakuten_mint,FALSE,FALSE,Column Level,,bup_stage.stg_rakuten,False
41,bup_stage.stg_rakuten_signature,bup_stage,stg_rakuten_signature,smartmoney_stg.stg_rakuten_signature_qetl,smartmoney_stg,stg_rakuten_signature_qetl,FALSE,FALSE,Column Level,,bup_stage.stg_rakuten_signature,False


In [12]:
df_dl_map.shape

(338, 12)

In [13]:
df_dl_map.sample(50)

,CTG Vertica Key,CTG Vertica AWS schema,CTG Vertica AWS table,Data Lake Key,Data Lake Schema,Data Lake Table,Same schema,Same table,Sync Type,Status,key,same_table_name
593,ctg_analytics.sub_clickstream_visitor_nonffa_f...,ctg_analytics,sub_clickstream_visitor_nonffa_first,tax_src.sub_clickstream_visitor_nonffa_first_g...,tax_src,sub_clickstream_visitor_nonffa_first_global,FALSE,FALSE,Column Level,,ctg_analytics.sub_clickstream_visitor_nonffa_f...,False
610,ctg_analytics_secure_src.trn_responsys_tt_skipped,ctg_analytics_secure_src,trn_responsys_tt_skipped,tax_src.src_responsys_turbotax_skipped,tax_src,src_responsys_turbotax_skipped,FALSE,FALSE,Column Level,,ctg_analytics_secure_src.trn_responsys_tt_skipped,False
322,consumer_source.src_document_service_expert,consumer_source,src_document_service_expert,tax_src.src_fdp_document_service_event,tax_src,src_fdp_document_service_event,FALSE,FALSE,Column Level,,consumer_source.src_document_service_expert,False
90,care_dwh.dim_ewfm_seg_code,care_dwh,dim_ewfm_seg_code,ent_ewfm_tcsdbowner_dwh.seg_code,ent_ewfm_tcsdbowner_dwh,seg_code,FALSE,FALSE,Column Level,,care_dwh.dim_ewfm_seg_code,False
715,mint_dwh.transaction_bankcc,mint_dwh,transaction_bankcc,smartmoney_dm.transaction_bankcc_full_load and...,smartmoney_dm,transaction_bankcc_full_load and transaction_b...,FALSE,FALSE,Column Level,,mint_dwh.transaction_bankcc,False
378,consumer_source.src_snap_qbse_total_subs_daily,consumer_source,src_snap_qbse_total_subs_daily,sbseg_dm.snap_qbse_total_subs_daily,sbseg_dm,snap_qbse_total_subs_daily,FALSE,FALSE,Column Level,,consumer_source.src_snap_qbse_total_subs_daily,False
432,ctg_analyst_layer.cases_uids,ctg_analyst_layer,cases_uids,tax_rpt.rpt_case_uid,tax_rpt,rpt_case_uid,FALSE,FALSE,Column Level,,ctg_analyst_layer.cases_uids,False
480,ctg_analyst_layer.inet_turbotax_api_details,ctg_analyst_layer,inet_turbotax_api_details,ivr_src.src_inet_turbotax_api_detail,ivr_src,src_inet_turbotax_api_detail,FALSE,FALSE,Column Level,,ctg_analyst_layer.inet_turbotax_api_details,False
293,consumer_source.src_braze_turbotax_users_canva...,consumer_source,src_braze_turbotax_users_canvas_conversion,tax_src.src_braze_turbotax_canvas_conversion,tax_src,src_braze_turbotax_canvas_conversion,FALSE,FALSE,Column Level,,consumer_source.src_braze_turbotax_users_canva...,False
584,ctg_analytics.lkp_item_alias_rank,ctg_analytics,lkp_item_alias_rank,tax_src.lkp_item_alias_rank/lkp_item_alias_ran...,tax_src,lkp_item_alias_rank/lkp_item_alias_rank_vw,FALSE,FALSE,Column Level,,ctg_analytics.lkp_item_alias_rank,False


#### Read vertica descriptions

In [14]:
df_vertica_data = read_gsheet(vertica_workbook, VERTICA_SHEET)

df_vertica_data.shape

(5137, 8)

In [15]:
df_vertica_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5137 entries, 0 to 5136
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   key          5137 non-null   object
 1   schema       5137 non-null   object
 2   table        5137 non-null   object
 3   column       5137 non-null   object
 4   title        5137 non-null   object
 5   description  5137 non-null   object
 6   tag          5137 non-null   object
 7   data file    5137 non-null   object
dtypes: object(8)
memory usage: 321.2+ KB


In [16]:
# remove tag = exclude - indicator that not the preferred description for rows with multiple descriptions
df_vertica_data = df_vertica_data[df_vertica_data['tag'] != 'exclude']
df_vertica_data.shape

(5103, 8)

In [17]:
# remove descriptions listed in description exception 
df_vertica_data = df_vertica_data[df_vertica_data['description'].isin(VERTICA_DESC_EXCEPTIONS) == False]
df_vertica_data.shape

(4411, 8)

In [18]:
df_vertica_data['desc_length'] = df_vertica_data['description'].str.len()

In [19]:
# df_vertica_data.sample(15)

#### Create unique list of Vertica column descriptions

In [20]:
df_vertica_column_desc = df_vertica_data[df_vertica_data['column'] != ""][['key','column','title','description','tag','desc_length']]

df_vertica_column_desc.sort_values(by='column', ascending = True, inplace=True)
df_vertica_column_desc.sort_values(by='desc_length', ascending = False, inplace=True)

df_vertica_column_desc


,key,column,title,description,tag,desc_length
3492,care_dwh.fact_sf_case.segment__c,segment__c,,"<table border=""0"" cellpadding=""0"" cellspacing=...",N/A,3598
3491,care_dwh.fact_sf_case.segment__c,segment__c,,"<table border=""0"" cellpadding=""0"" cellspacing=...",N/A,3598
3528,care_dwh.fact_sf_case.service_type,service_type,Type of call,"<table border=""0"" cellpadding=""0"" cellspacing...",N/A,1563
3527,care_dwh.fact_sf_case.service_type,service_type,Type of call,"<table border=""0"" cellpadding=""0"" cellspacing...",N/A,1563
1690,ctg_analytics_ws.agg_taxml_ytd.flag_disabled_o...,flag_disabled_or_blind,Flag Disabled Origin Blind,"As a person with a disability, you may qualif...",N/A,1519
...,...,...,...,...,...,...
4161,ctg_analytics_ws.schc_tax_profile_ty20_7216.tr...,travel,Travel,Travel,N/A,6
3458,ctg_analytics_ws.schc_tax_profile_ty20_7216.se...,se_tax,se_tax,se_tax,N/A,6
1257,ctg_analyst_layer.ctg_care_cct_interactions.dt,dt,Date,Date,N/A,4
1138,ctg_analyst_layer.agent_details_proficiency.date,date,Date,Date,N/A,4


In [21]:
column_keyword_list = list(df_vertica_column_desc['column'].value_counts().sort_index().index) 
column_single_keyword_list = []
column_short_single_keyword_list = []

for c in column_keyword_list:
    if c.find("_") == -1:
        column_single_keyword_list.append(c)
        
for c in column_single_keyword_list:
    if len(c) <= 10:
        column_short_single_keyword_list.append(c)
        
print(f"Keyword list count:   {len(column_keyword_list)}")
print(f"Single keyword count: {len(column_single_keyword_list)}")
print(f"Short keyword count:  {len(column_short_single_keyword_list)}")
print("")
print(column_short_single_keyword_list)

Keyword list count:   2539
Single keyword count: 303
Short keyword count:  252

['address', 'address2', 'agi', 'aht', 'alliance', 'amount', 'ani', 'authid', 'bu', 'business', 'cancelled', 'cardtype', 'caseid', 'casenumber', 'channel', 'checksum', 'childcare', 'cid', 'cid1', 'cid2', 'cid3', 'cid4', 'cid5', 'cid6', 'cid7', 'cid8', 'city', 'client', 'cmc', 'company', 'con', 'country', 'createdate', 'cypy', 'date', 'day', 'defected', 'definition', 'deleted', 'desktop', 'details', 'direction', 'dnis', 'domain', 'downvotes', 'dt', 'edition', 'effort', 'ein', 'email', 'email1', 'email2', 'email3', 'estore', 'ethnicity', 'evar01', 'evar02', 'evar03', 'evar04', 'evar05', 'evar06', 'evar07', 'evar08', 'evar09', 'evar10', 'evar11', 'evar12', 'evar13', 'evar14', 'evar15', 'evar16', 'evar17', 'evar18', 'evar19', 'evar20', 'evar21', 'evar22', 'evar23', 'evar24', 'evar25', 'evar26', 'evar27', 'evar28', 'evar29', 'evar30', 'evar31', 'evar32', 'evar33', 'evar34', 'evar35', 'evar36', 'evar37', 'evar38',

In [22]:
df_vertica_column_desc.shape

(4230, 6)

In [23]:
# prior to removing duplicates
# print(df_vertica_column_desc[['column','desc_length']].groupby('column').max().head(50))
# print(df_vertica_column_desc[['column','desc_length']].groupby('column').count().head(50))

In [24]:
df_vertica_column_desc.duplicated(subset=['column']).sum()

1691

In [25]:
df_vertica_column_desc.drop_duplicates(subset=['column'],inplace=True)
df_vertica_column_desc.shape

(2539, 6)

In [26]:
# cleaned-up of duplicates
# print(df_vertica_column_desc[['column','desc_length']].groupby('column').max().head(50))
# print(df_vertica_column_desc[['column','desc_length']].groupby('column').count().head(50))

In [27]:
df_vertica_column_desc.sort_values(by='column', ascending = True, inplace=True)
df_vertica_column_desc.reset_index
df_vertica_column_desc.head()

,key,column,title,description,tag,desc_length
0,ctg_analytics.fact_clickstream.ab_test_id,ab_test_id,Ab Test Identify,the Auth interaction with the product is a par...,N/A,141
1,care_dwh.fact_cc_standardized.abandon_flg,abandon_flg,Abandon Flag,1 or 0 to indicate if a contact that made it t...,N/A,347
4,ctg_analyst_layer.ctg_care_cct_interactions_vw...,abandoned_flg,Abandoned Flag,Indicates if a call was abandoned by the custo...,N/A,71
6,care_dwh.ewfm_cap_plan.abn_percent,abn_percent,,Percentage of calls not handled in the indicat...,N/A,58
7,ctg_analytics_ws.agg_taxml_ytd.aca_full_year_c...,aca_full_year_coverage,ACA Full Year Coverage,ACA Flag if you the user have the coverage for...,N/A,248


#### Create a unique list of Vertica table description

In [28]:
df_vertica_table_desc = df_vertica_data[(df_vertica_data['column'] == "") & ((df_vertica_data['table'] != "")) ][['key','table','title','description','tag','desc_length']]

df_vertica_table_desc.sort_values(by='table', ascending = True, inplace=True)
df_vertica_table_desc.sort_values(by='desc_length', ascending = False, inplace=True)

df_vertica_table_desc

,key,table,title,description,tag,desc_length
4955,ctg_analyst_layer.agent_details_proficiency,agent_details_proficiency,Agent Details Proficiency,The agent details proficiency query is the pr...,null,1427
5115,tax_published.ty20_full_service_master,ty20_full_service_master,Type 20 Full Service Master,<strong><u>Table Description: </u></strong> 1...,null,948
5039,ctg_analytics.fact_taxprep_session,fact_taxprep_session,,SOURCE: MATT \n\nThis table is meant to repre...,"[""compliance.ccpa"", ""product.turbo_tax""]",874
4976,ctg_analyst_layer.ctg_care_cases_voc,ctg_care_cases_voc,Case Surveys and Verbatims Table,"<u>Github link</u> <a href=""https://github.in...","[""compliance.ccpa"", ""product.turbo_tax""]",820
4978,ctg_analyst_layer.ctg_care_cct_interactions_vw,ctg_care_cct_interactions_vw,,ININ and Amazon Connect calls/interactions wi...,null,722
...,...,...,...,...,...,...
5095,ctg_analyst_layer.testing_analytics_master_auth,testing_analytics_master_auth,Testing Analytics Master Auth,Testing Analytics Master Auth,null,29
4986,ctg_analytics_ws.cy_turbo_cmc_master,cy_turbo_cmc_master,Turbo EM CMC Master,List of all Turbo EM CMC.,null,25
4957,ctg_analyst_layer.agent_proficiency_total,agent_proficiency_total,Agent Proficiency Total,Agent Proficiency Total,null,23
5081,consumer_source.src_pq_applicant,src_pq_applicant,Source Prequal Applicant,Prequal applicant data,null,22


In [29]:
df_vertica_table_desc.shape

(160, 6)

In [30]:
# df_vertica_table_desc.head(50)

In [31]:
df_vertica_table_desc.duplicated(subset=['table']).sum()

25

In [32]:
df_vertica_table_desc.drop_duplicates(subset=['table'],inplace=True)
df_vertica_table_desc.shape

(135, 6)

In [33]:
df_vertica_table_desc.sort_values(by='table', ascending = True, inplace=True)
df_vertica_table_desc.reset_index
df_vertica_table_desc.head()

,key,table,title,description,tag,desc_length
4954,care_dwh.agent_auth_corp_id_map,agent_auth_corp_id_map,Agent authentication id and corp id mapping table,This table contains the mapping of agents corp...,"[""compliance.ccpa"", ""product.turbo_tax""]",270
4955,ctg_analyst_layer.agent_details_proficiency,agent_details_proficiency,Agent Details Proficiency,The agent details proficiency query is the pr...,null,1427
4956,ctg_analyst_layer.agent_proficiency_exp_levels,agent_proficiency_exp_levels,Agent Proficiency,Agent Proficiency Measured by Call Volume,null,41
4957,ctg_analyst_layer.agent_proficiency_total,agent_proficiency_total,Agent Proficiency Total,Agent Proficiency Total,null,23
4958,ctg_analytics_ws.agg_auth_id_sku_selection,agg_auth_id_sku_selection,Aggreg Auth Identify Sku Selection,aggregate table that summarizes where users ch...,"[""compliance.ccpa"", ""product.turbo_tax""]",166


### read in Alation current data lake catalog 

From latest Alation data catalog export 

In [34]:
source_files = []

for f in os.listdir(CATALOG_LOCAL_PATH):
    if f[-3:] == "csv":
        source_files.append(f)
        
print(source_files)

['bud_clickstream_dwh_schema_19990_2104_2022-03-29T15-39-25-899169.csv', 'bup_dm_schema_20306_2104_2022-03-29T15-40-05-454383.csv', 'bup_src_schema_20307_2104_2022-03-29T15-43-51-419732.csv', 'cgdata_taxml_dwh_schema_16407_2104_2022-03-29T15-40-50-541437.csv', 'common_dm_schema_20479_2104_2022-03-29T15-45-32-002211.csv', 'ixp_dwh_schema_16459_2104_2022-03-29T15-44-58-163604.csv', 'smartmoney_dm_schema_19722_2104_2022-03-29T15-49-03-626123.csv', 'smartmoney_src_schema_20123_2104_2022-03-29T15-50-50-984401.csv', 'tax_dm_schema_19392_2104_2022-03-29T15-49-40-053659.csv', 'tax_rpt_schema_19891_2104_2022-03-29T15-54-20-533275.csv', 'tax_src_schema_19460_2104_2022-03-29T15-52-13-842602.csv', 'thrive_dwh_7216_restricted_schema_15534_2104_2022-03-29T16-00-26-992931.csv', 'thrive_dwh_schema_1551_2104_2022-03-29T16-06-46-408881.csv', 'ued_creditscore_dwh_schema_15280_2104_2022-03-29T16-01-57-526424.csv', 'ued_mint_dwh_schema_18258_2104_2022-03-29T16-02-44-977558.csv']


In [35]:
# create empty data frame 
df_alation_catalog = pd.DataFrame( 
                columns = ['key','title', 'description','steward:user','intuit_classification',
                           'schema','table','column','needs_description'])

df_alation_catalog['needs_description'] = df_alation_catalog['needs_description'].astype('bool')

df_alation_catalog.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   key                    0 non-null      object
 1   title                  0 non-null      object
 2   description            0 non-null      object
 3   steward:user           0 non-null      object
 4   intuit_classification  0 non-null      object
 5   schema                 0 non-null      object
 6   table                  0 non-null      object
 7   column                 0 non-null      object
 8   needs_description      0 non-null      bool  
dtypes: bool(1), object(8)
memory usage: 0.0+ bytes


In [36]:
# create Alation current catalog data frame
for f in source_files:
    # read catalog data file 
    source_file = os.path.join(CATALOG_LOCAL_PATH, f)
    
    df_temp = pd.read_csv(source_file,  usecols=['key','title','description','steward:user','intuit_classification'])
        
    key_values = df_temp['key'].tolist()

    # break out key into individual elements
    schema_names = []
    table_names = []
    column_names = []

    for i in range(0,len(key_values)):
        key_elements = key_values[i].split(".")

        schema_names.append(key_elements[0])
        if len(key_elements) >= 2:
            table_names.append(key_elements[1])
        else:
            table_names.append("")
        if len(key_elements) >= 3:
            column_names.append(key_elements[2])
        else:
            column_names.append("")

    df_temp['schema'] = schema_names
    df_temp['table'] = table_names
    df_temp['column'] = column_names
    
    # add needs description bolean
    df_temp['needs_description'] = df_temp['description'].isnull() 
    
    df_alation_catalog = pd.concat([df_alation_catalog, df_temp], ignore_index=True, sort=False)
    
df_alation_catalog.shape

(62763, 9)

In [37]:
df_alation_catalog[['key','schema','table','column','description']].sample(20)

,key,schema,table,column,description
32714,tax_src.src_taxml_historical.k1pw_rental_activ...,tax_src,src_taxml_historical,k1pw_rental_actively_participated,NaN
26913,tax_rpt.agg_user_care_case.max_datetime,tax_rpt,agg_user_care_case,max_datetime,NaN
19032,tax_dm.agg_auth_id_revenue.total_audit_defense...,tax_dm,agg_auth_id_revenue,total_audit_defense_revenue,<p>It is the revenue when the customer attach...
59256,ued_mint_dwh.form.mfatype,ued_mint_dwh,form,mfatype,NaN
23987,tax_rpt.agent_details_51346.nbr_auto_closed_cases,tax_rpt,agent_details_51346,nbr_auto_closed_cases,NaN
59376,ued_mint_dwh.credit_profile_ingest.creationtime,ued_mint_dwh,credit_profile_ingest,creationtime,NaN
36561,tax_src.src_fdp_document_service_event.year,tax_src,src_fdp_document_service_event,year,NaN
50678,thrive_dwh.cto_fdp_widget_service.providerid,thrive_dwh,cto_fdp_widget_service,providerid,NaN
18590,tax_dm.fact_financial_events.last_credit_card_...,tax_dm,fact_financial_events,last_credit_card_account_opened_date,NaN
55294,thrive_dwh.cg_turbotax_clickstream.sibling_eve...,thrive_dwh,cg_turbotax_clickstream,sibling_event_sender_versions,<p>Sibling Event Sender Versions -- where 'sib...


In [38]:
# filter out data frame to retain columns that need a description
df_alation_catalog = df_alation_catalog[(df_alation_catalog['needs_description']) & (df_alation_catalog['column'] != "")]

df_alation_catalog.shape

(51938, 9)

In [39]:
# remove columns with names in short names list 
df_alation_catalog = df_alation_catalog[df_alation_catalog['column'].isin(column_short_single_keyword_list) == False]
df_alation_catalog.shape

(48443, 9)

In [40]:
df_alation_catalog[['key','schema','table','column','description']].sample(20)

,key,schema,table,column,description
76,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,ecs_clickstream_turbotax_ty2021_v4,properties_custom_cf_action_value,NaN
11206,cgdata_taxml_dwh.taxml_bkp.stinfotp_felona,cgdata_taxml_dwh,taxml_bkp,stinfotp_felona,NaN
46167,thrive_dwh.cg_turbotax_clickstream_test.top_ev...,thrive_dwh,cg_turbotax_clickstream_test,top_event_sender_alias,NaN
10300,cgdata_taxml_dwh.taxml_delete_27thjan_bk.amoun...,cgdata_taxml_dwh,taxml_delete_27thjan_bk,amount_other_credits,NaN
51795,thrive_dwh.fdp_mint_global_clickstream_mobile....,thrive_dwh,fdp_mint_global_clickstream_mobile,post_first_hit_pagename,NaN
24863,tax_rpt.product_analytics_master_parity.start_...,tax_rpt,product_analytics_master_parity,start_sku_rollup_id,NaN
56118,thrive_dwh.cto_fdp_credential_offering_intent....,thrive_dwh,cto_fdp_credential_offering_intent,hive_timestamp,NaN
28432,tax_src.src_taxml.amount_taxable_ira,tax_src,src_taxml,amount_taxable_ira,NaN
8343,cgdata_taxml_dwh.taxml_delete.stinfodp_qaocn,cgdata_taxml_dwh,taxml_delete,stinfodp_qaocn,NaN
25730,tax_rpt.rpt_testing_analytics_master_auth2.mon...,tax_rpt,rpt_testing_analytics_master_auth2,monetization_take_max_upsell,NaN


### Prepare new definition data set 

Merge alation catalog with vertica data frames 

In [41]:
# create empty target data frame 
df_new_definitions = pd.DataFrame( 
                columns = ['key','schema','title','description','steward:user','intuit_classification'])

df_new_definitions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   key                    0 non-null      object
 1   schema                 0 non-null      object
 2   title                  0 non-null      object
 3   description            0 non-null      object
 4   steward:user           0 non-null      object
 5   intuit_classification  0 non-null      object
dtypes: object(6)
memory usage: 0.0+ bytes


#### Merge Vertica Column Definitions 

In [42]:
# print(df_alation_catalog.info())
# print(df_vertica_column_desc.info())

In [43]:
# merge alation catalog and vertica descriptions on column name
df_col_definitions = pd.merge(df_alation_catalog[['column','schema','key','title','steward:user','intuit_classification']], 
                        df_vertica_column_desc[['column','title','description']], on='column')

df_col_definitions.rename(columns={'title_x':'title'}, inplace=True)  

df_col_definitions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9141 entries, 0 to 9140
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   column                 9141 non-null   object
 1   schema                 9141 non-null   object
 2   key                    9141 non-null   object
 3   title                  7754 non-null   object
 4   steward:user           0 non-null      object
 5   intuit_classification  86 non-null     object
 6   title_y                9141 non-null   object
 7   description            9141 non-null   object
dtypes: object(8)
memory usage: 642.7+ KB


In [44]:
df_col_definitions['title'].isna().sum()

1387

In [45]:
df_col_definitions.fillna(value={'title': df_col_definitions['title_y']}, inplace=True)

In [46]:
df_col_definitions['title'].isna().sum()

0

In [47]:
df_col_definitions[['column','title','title_y']].sample(15)

,column,title,title_y
4163,offered_flg,Offered Flag,Offered Flag
9,tax_year,Tax Year,Tax Year
2617,spouse_military,Spouse Military,Spouse Military
2231,amount_taxable_ira,Amount Taxable Ira,Amount Taxable Ira
383,dwh_creation_time,Data Warehouse Creation Time,Dwh Creation Time
6815,site_area_id,Site Area Identify,Site Area Identify
6011,first_session_flag,First Session Flag,First Session Flag
2198,address_state,Address State,Address State
6853,trans_id,TRANS_ID,TRANS_ID
799,ingest_date,Ingest Date,Ingest Date


In [48]:
df_new_definitions = pd.concat([df_new_definitions, 
                               df_col_definitions[['key','schema','title','description','steward:user','intuit_classification']]], 
                               ignore_index = True)

In [49]:
df_new_definitions.fillna(value={'steward:user': ''}, inplace=True)
df_new_definitions.fillna(value={'intuit_classification': 'restricted'}, inplace=True)

df_new_definitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9141 entries, 0 to 9140
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   key                    9141 non-null   object
 1   schema                 9141 non-null   object
 2   title                  9141 non-null   object
 3   description            9141 non-null   object
 4   steward:user           9141 non-null   object
 5   intuit_classification  9141 non-null   object
dtypes: object(6)
memory usage: 428.6+ KB


#### Merge Vertica Table Definitions 

In [50]:
print(df_alation_catalog.info())
print(df_dl_map.info())
print(df_vertica_table_desc.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48443 entries, 15 to 62762
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   key                    48443 non-null  object
 1   title                  34571 non-null  object
 2   description            0 non-null      object
 3   steward:user           2 non-null      object
 4   intuit_classification  431 non-null    object
 5   schema                 48443 non-null  object
 6   table                  48443 non-null  object
 7   column                 48443 non-null  object
 8   needs_description      48443 non-null  bool  
dtypes: bool(1), object(8)
memory usage: 3.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 338 entries, 33 to 794
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CTG Vertica Key         338 non-null    object
 1   CTG Verti

In [51]:
# expand vertica table_desc to data lake table names
# match on vertica table key 
df_vertica_table_extended = pd.merge(df_vertica_table_desc,
                        df_dl_map[['key','CTG Vertica AWS schema','CTG Vertica AWS table',
                                   'Data Lake Key','Data Lake Schema','Data Lake Table']],
                        on='key')

# switch to DL table key 
df_vertica_table_extended['key'] = df_vertica_table_extended['Data Lake Key']

In [52]:
df_vertica_table_extended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   key                     32 non-null     object
 1   table                   32 non-null     object
 2   title                   32 non-null     object
 3   description             32 non-null     object
 4   tag                     32 non-null     object
 5   desc_length             32 non-null     int64 
 6   CTG Vertica AWS schema  32 non-null     object
 7   CTG Vertica AWS table   32 non-null     object
 8   Data Lake Key           32 non-null     object
 9   Data Lake Schema        32 non-null     object
 10  Data Lake Table         32 non-null     object
dtypes: int64(1), object(10)
memory usage: 3.0+ KB


In [53]:
df_vertica_table_extended.sort_values(by='key', ascending = True, inplace=True)

df_vertica_table_extended.head(32)

,key,table,title,description,tag,desc_length,CTG Vertica AWS schema,CTG Vertica AWS table,Data Lake Key,Data Lake Schema,Data Lake Table
13,ent_dwr_dwh.dw_as_standardized,fact_as_standardized,Agent States Call Data,Call Data pertaining to Agent States from Ama...,null,313,care_dwh,fact_as_standardized,ent_dwr_dwh.dw_as_standardized,ent_dwr_dwh,dw_as_standardized
16,ent_dwr_dwh.dw_cc_standardized,fact_cc_standardized,,There are key things to understand about this ...,null,550,care_dwh,fact_cc_standardized,ent_dwr_dwh.dw_cc_standardized,ent_dwr_dwh,dw_cc_standardized
21,ent_elm_dwh.elm_agent,sf_alm_agent,,WHAT: This table stores agent/pro's profile da...,null,226,care_dwh,sf_alm_agent,ent_elm_dwh.elm_agent,ent_elm_dwh,elm_agent
22,ent_elm_dwh.elm_agent_additional_info,sf_alm_agent_additional_info,,WHAT: This table stores agent/pro's profile da...,"[""compliance.ccpa"", ""product.turbo_tax""]",210,care_dwh,sf_alm_agent_additional_info,ent_elm_dwh.elm_agent_additional_info,ent_elm_dwh,elm_agent_additional_info
6,ent_ewfm_tcsdbowner_dwh.acd_login,dim_ewfm_acd_login,eWFM ACD Login Table,Table displaying ACD Logins to EMP_SK values. ...,null,183,care_dwh,dim_ewfm_acd_login,ent_ewfm_tcsdbowner_dwh.acd_login,ent_ewfm_tcsdbowner_dwh,acd_login
12,ent_ewfm_tcsdbowner_dwh.aud_det_seg,ewfm_det_seg,Ewfm Detail Segments,Lists segments in an employees Schedule over t...,null,234,care_dwh,ewfm_det_seg,ent_ewfm_tcsdbowner_dwh.aud_det_seg,ent_ewfm_tcsdbowner_dwh,aud_det_seg
7,ent_ewfm_tcsdbowner_dwh.seg_cat,dim_ewfm_seg_cat,eWFM Segment Category,Can be combined with the dim_ewfm_seg_code_cat...,null,172,care_dwh,dim_ewfm_seg_cat,ent_ewfm_tcsdbowner_dwh.seg_cat,ent_ewfm_tcsdbowner_dwh,seg_cat
8,ent_ewfm_tcsdbowner_dwh.seg_code_cat_assoc,dim_ewfm_seg_code_cat_assoc,ewfm SegCode/SegCat Association,Can be used to join the dim_ewfm_seg_code tabl...,null,141,care_dwh,dim_ewfm_seg_code_cat_assoc,ent_ewfm_tcsdbowner_dwh.seg_code_cat_assoc,ent_ewfm_tcsdbowner_dwh,seg_code_cat_assoc
9,ent_ewfm_tcsdbowner_dwh.tcs_login,dim_ewfm_tcs_login,eWFM TCS Logins,TCS Login Table - Can be used to associate the...,null,167,care_dwh,dim_ewfm_tcs_login,ent_ewfm_tcsdbowner_dwh.tcs_login,ent_ewfm_tcsdbowner_dwh,tcs_login
10,ent_ewfm_tcsdbowner_dwh.time_zone,dim_ewfm_time_zone,Dim Ewfm Time Zone,Can be used to tell which timezone employees/T...,null,141,care_dwh,dim_ewfm_time_zone,ent_ewfm_tcsdbowner_dwh.time_zone,ent_ewfm_tcsdbowner_dwh,time_zone


In [54]:
# merge alation catalog and vertica descriptions on table key name
df_table_definitions = pd.merge(df_alation_catalog[['key','schema','title','steward:user','intuit_classification']], 
                       df_vertica_table_extended[['key','title','description']], on='key')

df_table_definitions.rename(columns={'title_x':'title'}, inplace=True)

df_table_definitions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   key                    0 non-null      object
 1   schema                 0 non-null      object
 2   title                  0 non-null      object
 3   steward:user           0 non-null      object
 4   intuit_classification  0 non-null      object
 5   title_y                0 non-null      object
 6   description            0 non-null      object
dtypes: object(7)
memory usage: 0.0+ bytes


Processing stopped here - no new data lake table definition to bring in

### Read DSE catalog_set definitions 

This will be used as an exclude list - i.e. do not include object keys for which there is already an uploaded definition

In [55]:
# Initiatize cg_datasets data frame
df_cg_datasets = pd.DataFrame( 
                columns = ['key','data_file'])

# Read the list of Hive folders 
for d in os.listdir(GITHUB['LOCAL_PATH']):
    catalog_path = os.path.join(GITHUB['LOCAL_PATH'], d, GITHUB['DATA_CATALOG_FOLDER_NAME'])
    print("")
    if os.path.isdir(catalog_path):
        print(f"Processing content of {catalog_path}")
        f_count = 0
        for f in os.listdir(catalog_path):
            try:
                df_tmp = pd.read_csv(os.path.join(catalog_path, f),  usecols=['key'])
                df_tmp['data_file'] = f
#                 print(f"  adding {f} content")
                df_cg_datasets = pd.concat([df_cg_datasets, df_tmp], ignore_index=True, sort=True)
                f_count += 1
            except:
                pass
        print(f"   Read {f_count} input files")
        
    else:
        print(f"Skipping {catalog_path} - does not exist")


df_cg_datasets.shape


Processing content of S:/Development/github/intuit/cg_datasets/hive\bup_dm\datacatalog
   Read 34 input files

Processing content of S:/Development/github/intuit/cg_datasets/hive\bup_src\datacatalog
   Read 15 input files

Skipping S:/Development/github/intuit/cg_datasets/hive\care_rpt\datacatalog - does not exist

Skipping S:/Development/github/intuit/cg_datasets/hive\catax_src\datacatalog - does not exist

Skipping S:/Development/github/intuit/cg_datasets/hive\ca_tax_src\datacatalog - does not exist

Skipping S:/Development/github/intuit/cg_datasets/hive\cgan_ustax_published\datacatalog - does not exist

Skipping S:/Development/github/intuit/cg_datasets/hive\cg_dwh\datacatalog - does not exist

Processing content of S:/Development/github/intuit/cg_datasets/hive\common_dm\datacatalog
   Read 2 input files

Skipping S:/Development/github/intuit/cg_datasets/hive\digital_assist\datacatalog - does not exist

Processing content of S:/Development/github/intuit/cg_datasets/hive\fg_dwh\datac

(16522, 2)

In [56]:
df_cg_datasets['cg_datasets'] = True

df_cg_datasets.head(25)

,data_file,key,cg_datasets
0,bup_dm.config_etl_job.csv,bup_dm,True
1,bup_dm.config_etl_job.csv,bup_dm.config_etl_job,True
2,bup_dm.config_etl_job.csv,bup_dm.config_etl_job.schema_name,True
3,bup_dm.config_etl_job.csv,bup_dm.config_etl_job.max_processed_date,True
4,bup_dm.config_etl_job.csv,bup_dm.config_etl_job.etl_type_name,True
5,bup_dm.config_etl_job.csv,bup_dm.config_etl_job.job_load_status,True
6,bup_dm.config_etl_job.csv,bup_dm.config_etl_job.last_modified_time,True
7,bup_dm.config_etl_job.csv,bup_dm.config_etl_job.job_name,True
8,bup_dm.data_checker_report.csv,bup_dm.data_checker_report,True
9,bup_dm.data_checker_report.csv,bup_dm.data_checker_report.data_checker_name,True


### Remove columns already in cg datasets 

In [57]:
df_new_definitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9141 entries, 0 to 9140
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   key                    9141 non-null   object
 1   schema                 9141 non-null   object
 2   title                  9141 non-null   object
 3   description            9141 non-null   object
 4   steward:user           9141 non-null   object
 5   intuit_classification  9141 non-null   object
dtypes: object(6)
memory usage: 428.6+ KB


In [58]:
# use merge left join - substract 
# number of columns that will be remove
pd.merge(df_new_definitions,df_cg_datasets, on='key').shape

(2617, 8)

In [59]:
pd.merge(df_new_definitions,df_cg_datasets, on='key').head()

,key,schema,title,description,steward:user,intuit_classification,data_file,cg_datasets
0,tax_dm.agg_sessions_to_complete.tax_year,tax_dm,Tax Year,"<table border=""0"" cellpadding=""0"" cellspacing=...",,restricted,tax_dm.agg_sessions_to_complete.csv,True
1,tax_dm.helper_bot_like_agent_domain.tax_year,tax_dm,Tax Year,"<table border=""0"" cellpadding=""0"" cellspacing=...",,restricted,tax_dm.helper_bot_like_agent_domain.csv,True
2,tax_dm.dim_date_tax_year_season.tax_year,tax_dm,Tax Year,"<table border=""0"" cellpadding=""0"" cellspacing=...",,restricted,tax_dm.dim_date_tax_year_season.csv,True
3,tax_dm.prs.tax_year,tax_dm,Tax Year,"<table border=""0"" cellpadding=""0"" cellspacing=...",,restricted,tax_dm.prs_verbatim.csv,True
4,tax_dm.prs_verbatim.tax_year,tax_dm,Tax Year,"<table border=""0"" cellpadding=""0"" cellspacing=...",,restricted,tax_dm.prs_verbatim.csv,True


In [60]:
df_new_definitions_scrubbed = pd.merge(df_new_definitions,df_cg_datasets, on='key', how='left')
df_new_definitions_scrubbed['cg_datasets'].fillna(False, inplace=True)

df_new_definitions_scrubbed[['cg_datasets','key']].groupby('cg_datasets').count()

,key
cg_datasets,
False,6530
True,2617


In [61]:
df_new_definitions_scrubbed = df_new_definitions_scrubbed[~ df_new_definitions_scrubbed['cg_datasets']]
df_new_definitions_scrubbed.shape

(6530, 8)

In [62]:
# generate datafile names for export
key_list = df_new_definitions_scrubbed['key'].tolist()
# print(key_list)

data_file_list = []

for k in key_list:
    k_array = k.split(".")
    data_file_list.append(k_array[0]+"/"+GITHUB['DATA_CATALOG_FOLDER_NAME']+"/"+k_array[1]+\
                          "_"+GITHUB['CATALOG_FILE_NAME_SUFFIX']+".csv")
    
# print(data_file_list)

df_new_definitions_scrubbed['data_file'] = data_file_list

In [63]:
df_new_definitions_scrubbed.head()

,key,schema,title,description,steward:user,intuit_classification,data_file,cg_datasets
0,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"<table border=""0"" cellpadding=""0"" cellspacing=...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
1,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"<table border=""0"" cellpadding=""0"" cellspacing=...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
2,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"<table border=""0"" cellpadding=""0"" cellspacing=...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
3,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"<table border=""0"" cellpadding=""0"" cellspacing=...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
4,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"<table border=""0"" cellpadding=""0"" cellspacing=...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False


### Export new definitions to DSE GITHUB repo

In [64]:
# final data set
df_new_definitions_scrubbed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6530 entries, 0 to 9146
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   key                    6530 non-null   object
 1   schema                 6530 non-null   object
 2   title                  6530 non-null   object
 3   description            6530 non-null   object
 4   steward:user           6530 non-null   object
 5   intuit_classification  6530 non-null   object
 6   data_file              6530 non-null   object
 7   cg_datasets            6530 non-null   bool  
dtypes: bool(1), object(7)
memory usage: 414.5+ KB


In [65]:
df_new_definitions_scrubbed.duplicated(subset=['key']).sum()

0

In [66]:
# sort and add legacy tag to definition
df_new_definitions_scrubbed.sort_values(by='data_file', ascending = True, inplace=True)
df_new_definitions_scrubbed['description'] = "[Vertica Legacy] " + df_new_definitions_scrubbed['description'] 
df_new_definitions_scrubbed.head(25)

,key,schema,title,description,steward:user,intuit_classification,data_file,cg_datasets
6,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"[Vertica Legacy] <table border=""0"" cellpadding...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
2,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"[Vertica Legacy] <table border=""0"" cellpadding...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
9,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"[Vertica Legacy] <table border=""0"" cellpadding...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
4,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"[Vertica Legacy] <table border=""0"" cellpadding...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
1,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"[Vertica Legacy] <table border=""0"" cellpadding...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
7,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"[Vertica Legacy] <table border=""0"" cellpadding...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
3,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"[Vertica Legacy] <table border=""0"" cellpadding...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
5,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"[Vertica Legacy] <table border=""0"" cellpadding...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
0,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"[Vertica Legacy] <table border=""0"" cellpadding...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False
8,bud_clickstream_dwh.ecs_clickstream_turbotax_t...,bud_clickstream_dwh,Tax Year,"[Vertica Legacy] <table border=""0"" cellpadding...",,restricted,bud_clickstream_dwh/datacatalog/ecs_clickstrea...,False


In [67]:
# export as a single CSV file
df_export = df_new_definitions[['key','title','description','steward:user','intuit_classification']] 
df_export.to_csv('dl_new_definitions.csv', index=False)

In [69]:
# export into multiple CSV files, one per schema (i.e. key.split()[0]+"."+key.split()[1]+".csv")
# loop through export file list
csv_file_list = df_new_definitions_scrubbed['data_file'].unique().tolist()


for f in csv_file_list:
    # create table folder if not exists
    fx = f.split("/")
    f_path = os.path.join(GITHUB['LOCAL_PATH'],fx[0],fx[1])
#     print(f_path)
    os.makedirs(f_path,exist_ok=True)
    f_path = os.path.join(f_path,fx[2])    
    print(f"Exported: {f_path}")

    df_export = df_new_definitions_scrubbed[df_new_definitions_scrubbed['data_file'] == f][['key','title','description','steward:user','intuit_classification']] 
    df_export.to_csv(f_path, index=False)


Exported: S:/Development/github/intuit/cg_datasets/hive\bud_clickstream_dwh\datacatalog\ecs_clickstream_turbotax_ty2020_v2_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\bud_clickstream_dwh\datacatalog\ecs_clickstream_turbotax_ty2020_v3_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\bud_clickstream_dwh\datacatalog\ecs_clickstream_turbotax_ty2020_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\bud_clickstream_dwh\datacatalog\ecs_clickstream_turbotax_ty2021_v1_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\bud_clickstream_dwh\datacatalog\ecs_clickstream_turbotax_ty2021_v2_hive_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\bud_clickstream_dwh\datacatalog\ecs_clickstream_turbotax_ty2021_v2_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\bud_clickstream_dwh\datacatalog\ecs_clickstream_turbotax_ty2021_v3_hive_vertica-legacy.csv

Exported: S:/Development/github/intuit/cg_datasets/hive\smartmoney_src\datacatalog\account_bank_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\smartmoney_src\datacatalog\account_credit_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\smartmoney_src\datacatalog\account_fdp_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\smartmoney_src\datacatalog\account_investment_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\smartmoney_src\datacatalog\account_loan_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\smartmoney_src\datacatalog\account_property_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\smartmoney_src\datacatalog\account_realestate_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\smartmoney_src\datacatalog\account_unclassified_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive

Exported: S:/Development/github/intuit/cg_datasets/hive\tax_dm\datacatalog\fact_sf_b2c_case1_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_dm\datacatalog\fact_sf_b2c_case_history_raw_vnf_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_dm\datacatalog\fact_sf_b2c_contact_copy_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_dm\datacatalog\fact_sf_b2c_contact_verbatim_cgde50822_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_dm\datacatalog\fact_tax_digital_assistant_session_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_dm\datacatalog\fact_tax_w2_import_eligibility_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_dm\datacatalog\fact_tax_w2_metric_hist_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_dm\datacatalog\fact_tax_w2_metric_vertica-legacy.csv
Exported: S:/Development

Exported: S:/Development/github/intuit/cg_datasets/hive\tax_rpt\datacatalog\auth_analytics_base_new_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_rpt\datacatalog\auth_analytics_base_parity_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_rpt\datacatalog\auth_entitlement_daily_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_rpt\datacatalog\auth_sku_changes_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_rpt\datacatalog\ctg_care_call_auth_entitlement_bkp_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_rpt\datacatalog\ctg_care_call_auth_entitlement_snapshot_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_rpt\datacatalog\ctg_care_cases_voc_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_rpt\datacatalog\ctg_care_cct_interactions_vw_vertica-legacy.csv
Exported: S:/Development/g

Exported: S:/Development/github/intuit/cg_datasets/hive\tax_src\datacatalog\axc_ca_topics_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_src\datacatalog\axc_ca_users_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_src\datacatalog\axc_ca_votes_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_src\datacatalog\axc_us_analytic_events_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_src\datacatalog\axc_us_bookmarks_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_src\datacatalog\axc_us_communities_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_src\datacatalog\axc_us_helpful_votes_all_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_src\datacatalog\axc_us_post_attributes_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\tax_src\datacatalog\axc_us_posts_vert

...

Exported: S:/Development/github/intuit/cg_datasets/hive\thrive_dwh\datacatalog\cto_fdp_fdi_tax_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\thrive_dwh\datacatalog\cto_fdp_fpos_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\thrive_dwh\datacatalog\cto_fdp_fpp_apr_derivation_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\thrive_dwh\datacatalog\cto_fdp_non_tax_doc_extraction_quality_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\thrive_dwh\datacatalog\cto_fdp_profile_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\thrive_dwh\datacatalog\cto_fdp_provider_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\thrive_dwh\datacatalog\cto_fdp_tax_doc_extraction_quality_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\thrive_dwh\datacatalog\cto_fdp_transaction_test_vertica-legacy.csv
Exported: S:/Development/git

Exported: S:/Development/github/intuit/cg_datasets/hive\ued_credit_score_dwh\datacatalog\credit_inquiry_delete_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_credit_score_dwh\datacatalog\credit_inquiry_ingest_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_credit_score_dwh\datacatalog\credit_inquiry_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_credit_score_dwh\datacatalog\credit_monitoring_registration_delete_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_credit_score_dwh\datacatalog\credit_monitoring_registration_ingest_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_credit_score_dwh\datacatalog\credit_monitoring_registration_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_credit_score_dwh\datacatalog\credit_profile_delete_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\

Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\alert_delete_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\alert_ingest_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\alert_uncategorized_txn_delete_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\alert_uncategorized_txn_ingest_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\alert_uncategorized_txn_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\alert_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\alertrule_configuration_delete_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\alertrule_configuration_ingest_vertica-legacy.csv
Exported: S:/Develop

Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\notification_delete_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\notification_ingest_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\notification_jdbc_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\notification_tracking_delete_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\notification_tracking_ingest_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\notification_tracking_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\notification_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\offer_affiliate_delete_vertica-legacy.csv
Exported: S:/Development/g

Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\user_fdp_migration_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\user_goal_account_contribution_delete_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\user_goal_account_contribution_ingest_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\user_goal_account_contribution_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\user_goal_account_delete_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\user_goal_account_ingest_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\user_goal_account_vertica-legacy.csv
Exported: S:/Development/github/intuit/cg_datasets/hive\ued_mint_dwh\datacatalog\user_goal_action_delete_v